## **2. Prétraitement**
- Segmentation (phrases)
- Tokenization (mots)
- Étiquetage morphosyntaxique (POS Tagging) 
- (Lemmatisation - *sur la glace ; ça pourrait permettre d'éviter les faux négatifs dûs à des variations singulier/pluriel quand on essaie d'extraire les termes qui existent déjà dans la taxonomie*)
- Filtrage (stopwords)
- Extraction de termes complexes (MWE / n-grammes / segments répétés)
- Chunking / Filtrage par patrons syntaxiques (basés sur les patrons fréquents dans les MeSH)
- Extraction de collocations significatives (en fonction du Log-likelihood ratio)
- Extraction de concordances (KWIC) pour un ensemble de mots-clés d'intérêt
- Extraction de termes MeSH présents dans les données

### **Lire le corpus** 

In [95]:
import shutil, re, random
from os import listdir, chdir, path
from pathlib import Path
from pandas import *
import glob

import nltk
#nltk.download(['popular'])
from nltk.tokenize import RegexpTokenizer
tokenizer_re = RegexpTokenizer(r"\w\'|\w+")
from nltk import bigrams, trigrams, ngrams, everygrams
from nltk.probability import FreqDist


import treetaggerwrapper
tagger = treetaggerwrapper.TreeTagger(TAGLANG='fr')


from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.stats import binom, chi2

In [96]:
lng = 'fr'
acteur = 'ciusss_centreouest'
pdfs = False

if lng == 'fr':
    file = acteur +'.csv'
if lng == 'en':
    file = acteur + '_en.csv'

In [97]:
def lire_corpus(acteur = acteur, langue=lng):
    base_path = '../03-corpus/2-data/'
    
    folder_path = path.join(base_path, '1-' + langue, acteur)
    all_files = glob.glob(path.join(folder_path, "*.csv"))
    tags = [f.split('_')[1][:-4] for f in listdir(folder_path)]

    df = DataFrame()
    for f, tag in zip(all_files, tags):
        csv = read_csv(f, encoding='utf-8', sep=',')
        csv = csv[~csv["Address"].str.contains('pdf')] #  Problèmes 
        #csv = csv[~csv["Type"].str.contains('pdf', na=False)]
        #csv = csv['text']
        # Using DataFrame.insert() to add a column
        df = concat([df, csv]) [['Corpus', 'Sous-corpus', 'Address', 'Title', 'Type', 'text']]
        #df = concat([df, csv]) [['Corpus', 'Sous-corpus', 'Title', 'Type', 'text']]
    return df

def lire_corpus_csv(acteur):
    base_path = '../03-corpus/2-data/1-fr/'
    with open(base_path + acteur + '.csv', encoding='utf-8') as f:
        df = read_csv(f, encoding='utf-8', sep=',')
    return df


In [98]:
data = lire_corpus_csv(acteur) 
nb_docs = len(data)
print("On a un corpus de {} documents.".format(nb_docs))

On a un corpus de 730 documents.


In [99]:
base_path = '../03-corpus/2-data/'
folder_path = path.join(base_path, '1-' + lng, acteur)

data.to_csv(folder_path + '.csv')
data

,Unnamed: 0,Corpus,Sous-corpus,Address,Title,Type,text
0,0,ciusss_centreouest,Aire ouverte,https:////www.ciussscentreouest.ca//programmes...,CIUSSS du Centre-Ouest-de-l’Île-de-Montréal | ...,text//html,"Aire ouverte Si tu es en état de crise, compos..."
1,1,ciusss_centreouest,Aire ouverte,https:////www.ciussscentreouest.ca//programmes...,CIUSSS du Centre-Ouest-de-l’Île-de-Montréal | ...,text//html,Horaire et contact Horaire Appelle-nous au 514...
2,2,ciusss_centreouest,Aire ouverte,https:////www.ciussscentreouest.ca//programmes...,CIUSSS du Centre-Ouest-de-l’Île-de-Montréal | ...,text//html,Comités Comité des jeunes Les membres de ce co...
3,3,ciusss_centreouest,Aire ouverte,https:////www.ciussscentreouest.ca//programmes...,CIUSSS du Centre-Ouest-de-l’Île-de-Montréal | ...,text//html,Ressources Ressources jeunesse communautaires ...
4,4,ciusss_centreouest,Aire ouverte,https:////www.ciussscentreouest.ca//programmes...,CIUSSS du Centre-Ouest-de-l’Île-de-Montréal | ...,text//html,Espace partenaire Œuvrez-vous auprès de jeunes...
...,...,...,...,...,...,...,...
725,14,ciusss_centreouest,Vaccination,https:////www.ciussscentreouest.ca//nouvelles-...,CIUSSS du Centre-Ouest-de-l’Île-de-Montréal | ...,text//html,Nouvelles du CIUSSS PROJET PILOTE DE VACCINATI...
726,15,ciusss_centreouest,Vaccination,https:////www.ciussscentreouest.ca//nouvelles-...,CIUSSS du Centre-Ouest-de-l’Île-de-Montréal | ...,text//html,Nouvelles du CIUSSS Cliniques vaccination éphé...
727,16,ciusss_centreouest,Vaccination,https:////www.ciussscentreouest.ca//nouvelles-...,CIUSSS du Centre-Ouest-de-l’Île-de-Montréal | ...,text//html,Nouvelles du CIUSSS Clinique de rattrapage de ...
728,17,ciusss_centreouest,Vaccination,https:////www.ciussscentreouest.ca//nouvelles-...,Lire l'article complet,text//html,Nouvelles du CIUSSS Nouvelle clinique de vacci...


### **Nettoyage**

In [100]:
punct = '[!#$%&•►*+,;\/\\<=>?@[\]^_{|}~©«»—“”–—]'
spaces = '\s+'
postals = '([a-zA-Z]+\d+|\d+[a-zA-Z]+)+'
# phones = '\d{3}\s\d{3}-\d{4}' #très simple (trop)

text = [str(t).strip('\n').lower().replace('’', '\'') for t in data['text'].tolist()]
text = [re.sub(spaces, ' ', t) for t in text]
text = [re.sub(postals, ' STOP ', t) for t in text]
text = [re.sub(punct, ' STOP ', t) for t in text]
text = [t.replace("  ", " " ) for t in text]

### **Extraire un échantillon aléatoire**

Sinon, on n'arrive pas à traiter la totalité du corpus pour des raisons de performance

In [101]:
def sample_corpus(corpus, ratio):
    n = round(ratio * len(corpus))
    corpus = random.sample(text, n)
    print("On va travailler sur un échantillon correspondant à environ " + str(ratio * 100) + " % des documents du corpus, soit {} documents". format(len(corpus)))
    return " ".join(corpus)
    
corpus = sample_corpus(text, 1)

On va travailler sur un échantillon correspondant à environ 100 % des documents du corpus, soit 730 documents


### **Filtrage (MWE - stopwords formés de plusieurs tokens)**
Surtout pour filtrer les expressions relatives à l'architecture d'information / navigation Web

In [102]:
def filter_mwesw(corpus):
    file_mwesw = '../04-filtrage/mwe_stopwords.txt'
    with open (file_mwesw, 'r', encoding='utf-8') as f:
        mwe_sw = [t.lower().strip('\n') for t in f.readlines()]
    for mwe in mwe_sw:
        corpus = corpus.replace(mwe, ' STOP ').replace('  ', " ")
    return corpus

In [103]:
corpus = filter_mwesw(corpus)

### **Tokenisation / POS tagging** (TreeTagger)  
https://github.com/miotto/treetagger-python/blob/master/README.rst  
https://treetaggerwrapper.readthedocs.io/en/latest/

In [104]:
# Ici, on tokenise une première fois avec le Regex Tokenizer de NLTK pour voir combien de temps ça devrait 
# prendre au Tree Tagger pour tokeniser et tagger notre corpus
def tok(corpus):
    # Seulement les caractères alphabétiques
    tokens = tokenizer_re.tokenize(corpus)
    print("Avec le RegExpTokenizer, notre corpus contient {} tokens.".format(len(tokens)))
    temps = round(len(tokens) / 15000 / 60)
    print('Le POS tagging devrait prendre environ {} minutes.'.format(temps))
    return tokens

tok(corpus)[:10]

Avec le RegExpTokenizer, notre corpus contient 282010 tokens.
Le POS tagging devrait prendre environ 0 minutes.


['programme',
 'de',
 'prévention',
 'des',
 'chutes',
 'pied',
 'le',
 'programme',
 'pied',
 'est']

In [105]:
def tagging(corpus):
    output = []
    for t in tagger.tag_text(corpus):
        try: 
            output.append([t.split('\t')[0], t.split('\t')[1]])
        except Exception as e:
            output.append(('STOP', 'NAM'))

    return output


In [106]:
tagged = tagging(corpus)
tokens = [t[0] for t in tagged]

In [107]:
tagged

[['programme', 'NOM'],
 ['de', 'PRP'],
 ['prévention', 'NOM'],
 ['des', 'PRP:det'],
 ['chutes', 'NOM'],
 ['pied', 'NOM'],
 ['le', 'DET:ART'],
 ['programme', 'NOM'],
 ['pied', 'NOM'],
 ['est', 'VER:pres'],
 ['un', 'DET:ART'],
 ['programme', 'NOM'],
 ['de', 'PRP'],
 ['prévention', 'NOM'],
 ['des', 'PRP:det'],
 ['chutes', 'NOM'],
 ['intensif', 'ADJ'],
 ['gratuit', 'ADJ'],
 ["d'", 'PRP'],
 ['une', 'DET:ART'],
 ['durée', 'NOM'],
 ['de', 'PRP'],
 ['12', 'NUM'],
 ['semaines', 'NOM'],
 ['pour', 'PRP'],
 ['les', 'DET:ART'],
 ['personnes', 'NOM'],
 ['âgées', 'ADJ'],
 ['de', 'PRP'],
 ['65', 'NUM'],
 ['ans', 'NOM'],
 ['et', 'KON'],
 ['plus', 'ADV'],
 ['.', 'SENT'],
 ['le', 'DET:ART'],
 ['programme', 'NOM'],
 ['vise', 'VER:pres'],
 ['à', 'PRP'],
 ['améliorer', 'VER:infi'],
 ["l'", 'DET:ART'],
 ['équilibre', 'NOM'],
 ['STOP', 'INT'],
 ['la', 'DET:ART'],
 ['flexibilité', 'NOM'],
 ['et', 'KON'],
 ['la', 'DET:ART'],
 ['force', 'NOM'],
 ['physique', 'ADJ'],
 ['STOP', 'NOM'],
 ['à', 'PRP'],
 ['contribuer

### **Lemmatisation** (FrenchLefffLemmatizer)
Sur pause ; peut-être qu'il vaudrait mieux le faire plus tard (au moment d'identifier si chaque terme existe déjà dans la taxo ou non) afin d'éviter de complexifier toutes les autres étapes

In [108]:
# from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer
# lemmatizer = FrenchLefffLemmatizer()

# file_path = '../04-filtrage/mapping_treeTagger_lefff.csv'

# with open(file_path) as f:
#    csv = read_csv(f)

# treeTag = [term for term in csv['TreeTagger'].tolist()] 
# lefff = [term for term in csv['Lefff'].tolist()]

# mapping = {term : lefff[treeTag.index(term)] for term in treeTag}

# df_tagged = DataFrame(tagged, columns=['Token', 'Tag TreeTagger'])
# df_tagged['Tag Lefff'] = df_tagged['Tag TreeTagger'].map(mapping)
# df_tagged['Lemme'] = df_tagged['Token'] # S'il est pas capable de lemmatiser ensuite, on garde l'expression telle quelle

# tagged_dict = df_tagged.to_dict('records')

# # tokens = df_tagged['Token'].tolist()
# # tags = df_tagged['Tag Lefff'].tolist()
# # tagged_lefff = [(t, tg) for t,tg in zip(tokens, tags)]

# for term in tagged_dict:
#     if lemmatizer.lemmatize(word=term['Token'], pos=term['Tag Lefff']) == []:
#         term_l = lemmatizer.lemmatize(word=term['Token'])
    
#     elif type(lemmatizer.lemmatize(word=term['Token'], pos=term['Tag Lefff'])) == str:
#         term_l  = lemmatizer.lemmatize(word=term['Token'], pos=term['Tag Lefff'])

#     else:
#         term_l = lemmatizer.lemmatize(word=term['Token'], pos=term['Tag Lefff'])[0][0]

#     term['Lemme'] = term_l

# tagged = DataFrame(tagged_dict)
# tagged = tagged.drop(columns=['Tag Lefff']).values.tolist()

#### **Mapping POS Tags** (FRMG)
On doit mapper les étiquettes morphosyntaxiques du TreeTagger à celles que prend le lemmatiseur (celles issues de FRMG)
#http://alpage.inria.fr/frmgwiki/content/tagset-frmg

In [109]:
# file_path = '../04-filtrage/mapping_treeTagger_lefff.csv'

# with open(file_path) as f:
#    csv = read_csv(f)

# treeTag = [term for term in csv['TreeTagger'].tolist()] 
# lefff = [term for term in csv['Lefff'].tolist()]

# mapping = {term : lefff[treeTag.index(term)] for term in treeTag}

In [110]:
# df_tagged = DataFrame(tagged, columns=['Token', 'Tag TreeTagger'])
# df_tagged['Tag Lefff'] = df_tagged['Tag TreeTagger'].map(mapping)
# df_tagged['Lemme'] = df_tagged['Token'] # S'il est pas capable de lemmatiser ensuite, on garde l'expression telle quelle

In [111]:
# tagged_dict = df_tagged.to_dict('records')



### **Collocations / Phrases / N-Grammes (MWE)**
https://www.kaggle.com/code/alvations/n-gram-language-model-with-nltk/notebook  

In [115]:
def extr_ngrams(tagged):
    ngrammes= list(everygrams(tagged, min_len=2, max_len=8))
    print("Avant filtrage, on a {} ngrammes.".format(len(ngrammes)))
    return ngrammes

In [116]:
ngrammes = extr_ngrams(tagged)
#ngrammes_lem = extr_ngrams(tagged_lem)

Avant filtrage, on a 2029503 ngrammes.


In [117]:
ngrammes

[(['programme', 'NOM'], ['de', 'PRP']),
 (['programme', 'NOM'], ['de', 'PRP'], ['prévention', 'NOM']),
 (['programme', 'NOM'],
  ['de', 'PRP'],
  ['prévention', 'NOM'],
  ['des', 'PRP:det']),
 (['programme', 'NOM'],
  ['de', 'PRP'],
  ['prévention', 'NOM'],
  ['des', 'PRP:det'],
  ['chutes', 'NOM']),
 (['programme', 'NOM'],
  ['de', 'PRP'],
  ['prévention', 'NOM'],
  ['des', 'PRP:det'],
  ['chutes', 'NOM'],
  ['pied', 'NOM']),
 (['programme', 'NOM'],
  ['de', 'PRP'],
  ['prévention', 'NOM'],
  ['des', 'PRP:det'],
  ['chutes', 'NOM'],
  ['pied', 'NOM'],
  ['le', 'DET:ART']),
 (['programme', 'NOM'],
  ['de', 'PRP'],
  ['prévention', 'NOM'],
  ['des', 'PRP:det'],
  ['chutes', 'NOM'],
  ['pied', 'NOM'],
  ['le', 'DET:ART'],
  ['programme', 'NOM']),
 (['de', 'PRP'], ['prévention', 'NOM']),
 (['de', 'PRP'], ['prévention', 'NOM'], ['des', 'PRP:det']),
 (['de', 'PRP'], ['prévention', 'NOM'], ['des', 'PRP:det'], ['chutes', 'NOM']),
 (['de', 'PRP'],
  ['prévention', 'NOM'],
  ['des', 'PRP:det'],

### **Extraction des patrons syntaxiques**

In [118]:
def extract_patterns(ngrammes):
    patterns = []
    for ng in ngrammes:
        phrase = tuple([t[0] for t in ng])
        pattern = [t[1] for t in ng]
        patterns.append([phrase, pattern])
    return patterns

In [119]:
phrases = extract_patterns(ngrammes)
# phrases_lem = extract_patterns(ngrammes_lem)

In [120]:
# def freq(phrases):
#     return FreqDist([" ".join(t[0]).replace("' ", "'") for t in phrases])

# def freq(phrases):
#     return FreqDist([t[0] for t in phrases])

In [121]:
frequencies = FreqDist(everygrams(tokens, min_len=1, max_len=8))

In [122]:
frequencies

FreqDist({('STOP',): 21812, ('de',): 15000, ('.',): 8077, ('la',): 7403, ('et',): 6777, ('des',): 5635, ('à',): 5181, ('le',): 4629, ('les',): 4561, ("l'",): 4283, ...})

### **Filtrage** 
On retire les n-grammes qui débutent ou se terminent par un stopword (antidictionnaire)

In [123]:
# Importer l'antidictionnaire pour filtrer les données

# # Stopwords lemmatisés
# file_path = '../04-filtrage/stopwords_lemmatized.txt'
# with open(file_path, 'r', encoding="utf-8") as f:
#     stopwords_lemmatized = [w.strip('\n').lower() for w in f.readlines()]

# Stopwords fréquents en français (non lemmatisés)
file_path = "../04-filtrage/stopwords.txt"
with open(file_path, 'r', encoding="utf-8") as f:
    stopwords = [t.lower().strip('\n') for t in f.readlines()]


# Stopwords fréquents en anglais (non lemmatisés)
file_path = '../04-filtrage/stop_words_english.txt'
with open(file_path, 'r', encoding="utf-8") as f:
    stopwords += [t.lower().strip('\n') for t in f.readlines()]

*LONG*

In [124]:
def filtrer_stopwords(x): # On s'assure aussi que le premier terme du ngramme est un NOM pour avoir des syntagmes nominaux
    if lng == 'en':
        nom = 'NN'
    if lng == 'fr':
        nom = 'NOM'
    return [term for term in x if not 'STOP' in term[0] and not term[0][0] in stopwords and not term[0][-1] in stopwords \
        and not 'NUM' in term[1] and term[1][0] == nom and not '.' in term[0] and not '-' in term[0] and not ':' in term[0]\
        
        # Une parenthèse fermante peut juste se trouver comme dernier token
        # Si une parenthèse est ouverte, elle doit aussi être fermée (et vice versa)
        and not ')' in term[0][:-1] and not ('(' in term[0] and not ')' in term[0]) \
        and not (')' in term[0] and not '(' in term[0])]

In [125]:
phrases = filtrer_stopwords(phrases)

On retire les n-grammes qui débutent ou se terminent par un token dont la longueur est inférieure à 2 caractères ou supérieure à 18 caractères

In [126]:
def filter_len(x):
    return [term for term in x if \
        (len(term[0][0]) > 2 or term[0][0] == '(')  and (len(term[0][-1]) > 2 or term[0][-1] == ')') and \
        len(term[0][0]) < 18 and len(term[0][-1]) < 18]

phrases = filter_len(phrases)

In [127]:
phrases

[[('programme', 'de', 'prévention'), ['NOM', 'PRP', 'NOM']],
 [('programme', 'de', 'prévention', 'des', 'chutes'),
  ['NOM', 'PRP', 'NOM', 'PRP:det', 'NOM']],
 [('programme', 'de', 'prévention', 'des', 'chutes', 'pied'),
  ['NOM', 'PRP', 'NOM', 'PRP:det', 'NOM', 'NOM']],
 [('programme',
   'de',
   'prévention',
   'des',
   'chutes',
   'pied',
   'le',
   'programme'),
  ['NOM', 'PRP', 'NOM', 'PRP:det', 'NOM', 'NOM', 'DET:ART', 'NOM']],
 [('prévention', 'des', 'chutes'), ['NOM', 'PRP:det', 'NOM']],
 [('prévention', 'des', 'chutes', 'pied'), ['NOM', 'PRP:det', 'NOM', 'NOM']],
 [('prévention', 'des', 'chutes', 'pied', 'le', 'programme'),
  ['NOM', 'PRP:det', 'NOM', 'NOM', 'DET:ART', 'NOM']],
 [('prévention', 'des', 'chutes', 'pied', 'le', 'programme', 'pied'),
  ['NOM', 'PRP:det', 'NOM', 'NOM', 'DET:ART', 'NOM', 'NOM']],
 [('chutes', 'pied'), ['NOM', 'NOM']],
 [('chutes', 'pied', 'le', 'programme'), ['NOM', 'NOM', 'DET:ART', 'NOM']],
 [('chutes', 'pied', 'le', 'programme', 'pied'),
  [

On retire les n-grammes qui apparaissent moins de 5 fois dans le corpus

In [128]:
def filter_freq(x):
    return [term for term in x if frequencies[tuple(term[0])] > 5]

phrases = filter_freq(phrases)

In [129]:
phrases

[[('programme', 'de', 'prévention'), ['NOM', 'PRP', 'NOM']],
 [('programme', 'de', 'prévention', 'des', 'chutes'),
  ['NOM', 'PRP', 'NOM', 'PRP:det', 'NOM']],
 [('prévention', 'des', 'chutes'), ['NOM', 'PRP:det', 'NOM']],
 [('pied', 'le', 'programme'), ['NOM', 'DET:ART', 'NOM']],
 [('programme', 'pied'), ['NOM', 'NOM']],
 [('programme', 'de', 'prévention'), ['NOM', 'PRP', 'NOM']],
 [('programme', 'de', 'prévention', 'des', 'chutes'),
  ['NOM', 'PRP', 'NOM', 'PRP:det', 'NOM']],
 [('prévention', 'des', 'chutes'), ['NOM', 'PRP:det', 'NOM']],
 [('séances', 'de', 'groupe'), ['NOM', 'PRP', 'NOM']],
 [('professionnel', 'de', 'la', 'santé'), ['NOM', 'PRP', 'DET:ART', 'NOM']],
 [('programme', 'pied'), ['NOM', 'NOM']],
 [('programme', 'pied'), ['NOM', 'NOM']],
 [('clsc', 'côte-des-neiges'), ['NOM', 'ADJ']],
 [('centre', 'communautaire'), ['NOM', 'ADJ']],
 [('diminution', 'de', 'la', 'contamination'),
  ['NOM', 'PRP', 'DET:ART', 'NOM']],
 [('diminution', 'de', 'la', 'contamination', 'entre', 'emp

In [130]:
#phrases = [[" ".join(term[0]).replace("' ", "'"), " ".join(term[1])] for term in phrases]
phrases = [[term[0], " ".join(term[1])] for term in phrases]
# phrases_lemmatized = filtrer_phrases(phrases_lemmatized, freq_lemmatized)

In [131]:
for phrase in phrases:
    phrase.append(frequencies[tuple(phrase[0])])

In [132]:
print("Après filtrage, on a {} occurrences de ngrammes.".format(len(phrases))) 

Après filtrage, on a 26867 occurrences de ngrammes.


In [133]:
phrases[:15]

[[('programme', 'de', 'prévention'), 'NOM PRP NOM', 12],
 [('programme', 'de', 'prévention', 'des', 'chutes'),
  'NOM PRP NOM PRP:det NOM',
  9],
 [('prévention', 'des', 'chutes'), 'NOM PRP:det NOM', 26],
 [('pied', 'le', 'programme'), 'NOM DET:ART NOM', 6],
 [('programme', 'pied'), 'NOM NOM', 9],
 [('programme', 'de', 'prévention'), 'NOM PRP NOM', 12],
 [('programme', 'de', 'prévention', 'des', 'chutes'),
  'NOM PRP NOM PRP:det NOM',
  9],
 [('prévention', 'des', 'chutes'), 'NOM PRP:det NOM', 26],
 [('séances', 'de', 'groupe'), 'NOM PRP NOM', 10],
 [('professionnel', 'de', 'la', 'santé'), 'NOM PRP DET:ART NOM', 53],
 [('programme', 'pied'), 'NOM NOM', 9],
 [('programme', 'pied'), 'NOM NOM', 9],
 [('clsc', 'côte-des-neiges'), 'NOM ADJ', 39],
 [('centre', 'communautaire'), 'NOM ADJ', 9],
 [('diminution', 'de', 'la', 'contamination'), 'NOM PRP DET:ART NOM', 6]]

In [134]:
DataFrame(phrases, columns=["Expression", "Patron syntaxique", "Fréquence"])

phrases

[[('programme', 'de', 'prévention'), 'NOM PRP NOM', 12],
 [('programme', 'de', 'prévention', 'des', 'chutes'),
  'NOM PRP NOM PRP:det NOM',
  9],
 [('prévention', 'des', 'chutes'), 'NOM PRP:det NOM', 26],
 [('pied', 'le', 'programme'), 'NOM DET:ART NOM', 6],
 [('programme', 'pied'), 'NOM NOM', 9],
 [('programme', 'de', 'prévention'), 'NOM PRP NOM', 12],
 [('programme', 'de', 'prévention', 'des', 'chutes'),
  'NOM PRP NOM PRP:det NOM',
  9],
 [('prévention', 'des', 'chutes'), 'NOM PRP:det NOM', 26],
 [('séances', 'de', 'groupe'), 'NOM PRP NOM', 10],
 [('professionnel', 'de', 'la', 'santé'), 'NOM PRP DET:ART NOM', 53],
 [('programme', 'pied'), 'NOM NOM', 9],
 [('programme', 'pied'), 'NOM NOM', 9],
 [('clsc', 'côte-des-neiges'), 'NOM ADJ', 39],
 [('centre', 'communautaire'), 'NOM ADJ', 9],
 [('diminution', 'de', 'la', 'contamination'), 'NOM PRP DET:ART NOM', 6],
 [('diminution', 'de', 'la', 'contamination', 'entre', 'employés'),
  'NOM PRP DET:ART NOM PRP NOM',
  6],
 [('contamination', '

In [135]:
print("Après filtrage, on a {} ngrammes uniques.".format(len(DataFrame(phrases).drop_duplicates())))

Après filtrage, on a 2689 ngrammes uniques.


### **Filtrage (Patrons syntaxiques)**  
Lossio-Ventura, J. A., Jonquet, C., Roche, M., & Teisseire, M. (2014). Biomedical Terminology Extraction : A new combination of Statistical and Web Mining Approaches. 421. https://hal-lirmm.ccsd.cnrs.fr/lirmm-01056598

On veut aller extraire les structures syntaxiques les plus courantes dans les MeSH pour filtrer notre corpus selon celles-ci (inspiré de la méthodologie de l'article ci-dessus ; voir le Notebook *Mesh_extract.ipynb*). Pour ce faire, nous allons donc ne sélectionner que les ngrammes qui y correspondent. 

In [136]:
file_patterns = '../04-filtrage/MeSH/mesh_patterns-fr.csv'

with open (file_patterns, 'r') as f:
    patterns = read_csv(f)
    patterns = patterns['Structure'].tolist() # Pour prendre les structures syntaxiques attestées dans les MeSH

In [137]:
def filter_patterns(phrases):
    return [t for t in phrases if t[1] in patterns and not 'NUM' in t[1]] # and not 'NOM NOM' in t[1]
# terms_lemmatized = [t for t in phrases_lemmatized if t[1] in patterns]

In [138]:
terms = filter_patterns(phrases)
#terms = phrases

In [139]:
terms

[[('programme', 'de', 'prévention'), 'NOM PRP NOM', 12],
 [('programme', 'de', 'prévention', 'des', 'chutes'),
  'NOM PRP NOM PRP:det NOM',
  9],
 [('prévention', 'des', 'chutes'), 'NOM PRP:det NOM', 26],
 [('pied', 'le', 'programme'), 'NOM DET:ART NOM', 6],
 [('programme', 'pied'), 'NOM NOM', 9],
 [('programme', 'de', 'prévention'), 'NOM PRP NOM', 12],
 [('programme', 'de', 'prévention', 'des', 'chutes'),
  'NOM PRP NOM PRP:det NOM',
  9],
 [('prévention', 'des', 'chutes'), 'NOM PRP:det NOM', 26],
 [('séances', 'de', 'groupe'), 'NOM PRP NOM', 10],
 [('professionnel', 'de', 'la', 'santé'), 'NOM PRP DET:ART NOM', 53],
 [('programme', 'pied'), 'NOM NOM', 9],
 [('programme', 'pied'), 'NOM NOM', 9],
 [('clsc', 'côte-des-neiges'), 'NOM ADJ', 39],
 [('centre', 'communautaire'), 'NOM ADJ', 9],
 [('diminution', 'de', 'la', 'contamination'), 'NOM PRP DET:ART NOM', 6],
 [('diminution', 'de', 'la', 'contamination', 'entre', 'employés'),
  'NOM PRP DET:ART NOM PRP NOM',
  6],
 [('contamination', '

In [140]:
print("Le filtrage syntaxique élimine environ {} % des termes".format(round((len(phrases) - len(terms)) / len(phrases) * 100)))
print("On avait {} ngrammes, ".format(len(phrases)) + "on en a maintenant {}.".format(len(terms)))

Le filtrage syntaxique élimine environ 18 % des termes
On avait 26867 ngrammes, on en a maintenant 22079.


In [141]:
# import pandas as pd

# def extract_terms(liste_terms):
#     file_path = '../04-filtrage/output/'
#     tab = pd.DataFrame(terms, columns= ["Expression", "Structure syntaxique", "Fréquence"]).drop_duplicates(subset='Expression', keep="last")
#     tab.sort_values(["Fréquence"], 
#                         axis=0,
#                         ascending=[False], 
#                         inplace=True)

#     return 

#     # file_path = path.join(file_path, tag, tag)                    
#     # tab.to_csv(file_path + '_terms.csv')

# extract_terms(terms)
# #extract_terms(terms_lemmatized)

In [142]:
for phrase in terms:
    phrase[0] = tuple(phrase[0])


terms_patterns = DataFrame(terms, columns = ["Expression", "Structure syntaxique", "Fréquence"])
terms_patterns = terms_patterns.to_dict('records')
dict_patterns = {}
for term in terms_patterns:
     exp = term['Expression']
     pattern = term['Structure syntaxique']
     dict_patterns[exp] = pattern

In [143]:
terms_patterns

[{'Expression': ('programme', 'de', 'prévention'),
  'Structure syntaxique': 'NOM PRP NOM',
  'Fréquence': 12},
 {'Expression': ('programme', 'de', 'prévention', 'des', 'chutes'),
  'Structure syntaxique': 'NOM PRP NOM PRP:det NOM',
  'Fréquence': 9},
 {'Expression': ('prévention', 'des', 'chutes'),
  'Structure syntaxique': 'NOM PRP:det NOM',
  'Fréquence': 26},
 {'Expression': ('pied', 'le', 'programme'),
  'Structure syntaxique': 'NOM DET:ART NOM',
  'Fréquence': 6},
 {'Expression': ('programme', 'pied'),
  'Structure syntaxique': 'NOM NOM',
  'Fréquence': 9},
 {'Expression': ('programme', 'de', 'prévention'),
  'Structure syntaxique': 'NOM PRP NOM',
  'Fréquence': 12},
 {'Expression': ('programme', 'de', 'prévention', 'des', 'chutes'),
  'Structure syntaxique': 'NOM PRP NOM PRP:det NOM',
  'Fréquence': 9},
 {'Expression': ('prévention', 'des', 'chutes'),
  'Structure syntaxique': 'NOM PRP:det NOM',
  'Fréquence': 26},
 {'Expression': ('séances', 'de', 'groupe'),
  'Structure syntax

### **Filtrage (Collocations statistiquement significatives)** Log-Likelihood Ratio

[Notebook - Collocation extraction methodologies compared](https://notebooks.githubusercontent.com/view/ipynb?azure_maps_enabled=false&browser=chrome&color_mode=auto&commit=33868e847376764d7733cd958986c88dedfaec97&device=unknown&enc_url=68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f746f64642d636f6f6b2f4d4c2d596f752d43616e2d5573652f333338363865383437333736373634643737333363643935383938366338386465646661656339372f70726f626162696c69737469635f6c616e67756167655f6d6f64656c696e672f636f6c6c6f636174696f6e5f65787472616374696f6e732e6970796e62&enterprise_enabled=false&logged_in=false&nwo=todd-cook%2FML-You-Can-Use&path=probabilistic_language_modeling%2Fcollocation_extractions.ipynb&platform=android&repository_id=167140788&repository_type=Repository&version=102)

On applique un test d'hypothèse statistique aux n-grammes sur lesquels une probabilité a été mesurée (Log-likelihood ratio) - seuls les n-grammes dont le test est significatif seront conservés.
On considère que l'apparition de ces collocations dans notre corpus n'est pas dûe au hasard.

In [144]:
def loglikelihood_ratio(c_prior, c_n, c_ngram, N):
    """
    Compute the ratio of two hypotheses of likelihood and return the ratio.
    The formula here and test verification values are taken from 
    Manning & Schūtze _Foundations of Statistical Natural Language Processing_ p.172-175
    Parameters:
    c_prior: count of word 1 if bigrams or count of [w1w2 .. w(n-1)] if ngram
    c_n : count of word 2 if bigrams or count of wn if ngram
    c12: count of bigram (w1, w2) if bigram or count of ngram if ngram
    N: the number of words in the corpus
    """

    p = c_n / N
    p1 = c_ngram / c_prior
    p2 = (c_n - c_ngram) / (N - c_prior)   
    # We proactively trap a runtimeWarning: divide by zero encountered in log,
    # which may occur with extreme collocations
    import warnings
    with warnings.catch_warnings(): # this will reset our filterwarnings setting
        warnings.filterwarnings('error')
        try:
            return (np.log(binom.pmf(c_ngram, c_prior, p)) 
                    + np.log(binom.pmf(c_n - c_ngram, N - c_prior, p)) 
                    - np.log(binom.pmf(c_ngram, c_prior, p1) )
                    - np.log(binom.pmf(c_n - c_ngram, N - c_prior, p2)))             
        except Warning:
            return np.inf 

In [145]:
len_prior = len(terms)
print("Au départ, on a {} ngrammes.".format(len_prior))

Au départ, on a 22079 ngrammes.


In [146]:
frequencies

FreqDist({('STOP',): 21812, ('de',): 15000, ('.',): 8077, ('la',): 7403, ('et',): 6777, ('des',): 5635, ('à',): 5181, ('le',): 4629, ('les',): 4561, ("l'",): 4283, ...})

In [147]:
# Pour le calcul des probabilités, on a besoin de traiter séparément les ngrammes selon la valeur de n
N = len(tokens)
fd_tokens = nltk.FreqDist(tokens)

In [148]:
def llr_ngrammes(terms):
    llr = []

    ngrammes = [term[0] for term in terms]
        
    for t in ngrammes:
        if len(t) == 1:
            try:
                llr.append({'Terme' : str(t[0]), 'Structure syntaxique': dict_patterns[t], 'Fréquence (TF)' : fd_tokens[str(t[0])], 'LLR': '-', 'p-value': '-'})
            except Exception as e:
                print(t, str(e))
        else:
            c_prior = frequencies[t[:-1]] # Antécédent = P(w1w2..w_n-1) (si on considère que P(w1w2...wn) = P(wn) | P(w1w2...w_n-1)
            c_n = fd_tokens[t[-1]]     # Dernier mot du ngramme  P(wn)
            c_ngram = frequencies[t]             # Le ngramme lui-même P(w1w2w3..wn)

            try:
                res = -2 * loglikelihood_ratio(c_prior, c_n, c_ngram, N)
                p_value = chi2.sf(res, 1) # 1 degrees of freedom

                if p_value < 0.001 or (res == float('-inf')):
                    #llr.append({'Collocation' : " ".join(t).replace("' ", "'").replace("( ", "(").replace(" )", ")"), 'Structure syntaxique': dict_patterns[" ".join(t).replace("' ", "'")], 'Fréquence' : c_ngram, 'LLR': res, 'p-value': p_value})
                    llr.append({'Terme' : t, 'Structure syntaxique': dict_patterns[t], 'Fréquence (TF)' : c_ngram, 'LLR': res, 'p-value': p_value})
            
            except Exception as e:
                print(t, str(e))
            
    return llr


In [149]:
significant_coll = llr_ngrammes(terms)
#terms = [{'Collocation' : t[0], 'Structure syntaxique': t[1], 'Fréquence' : t[2]} for t in terms] # Si on veut pas appliquer le LLR

In [150]:
significant_coll


[{'Terme': ('programme', 'de', 'prévention'),
  'Structure syntaxique': 'NOM PRP NOM',
  'Fréquence (TF)': 12,
  'LLR': 78.27795101581269,
  'p-value': 8.951546652665308e-19},
 {'Terme': ('programme', 'de', 'prévention', 'des', 'chutes'),
  'Structure syntaxique': 'NOM PRP NOM PRP:det NOM',
  'Fréquence (TF)': 9,
  'LLR': 164.939668829249,
  'p-value': 9.429511766350728e-38},
 {'Terme': ('prévention', 'des', 'chutes'),
  'Structure syntaxique': 'NOM PRP:det NOM',
  'Fréquence (TF)': 26,
  'LLR': 408.1978478896003,
  'p-value': 9.045046040378483e-91},
 {'Terme': ('pied', 'le', 'programme'),
  'Structure syntaxique': 'NOM DET:ART NOM',
  'Fréquence (TF)': 6,
  'LLR': 71.66519543048653,
  'p-value': 2.549908217606719e-17},
 {'Terme': ('programme', 'pied'),
  'Structure syntaxique': 'NOM NOM',
  'Fréquence (TF)': 9,
  'LLR': 66.07781911844478,
  'p-value': 4.334664604245329e-16},
 {'Terme': ('programme', 'de', 'prévention'),
  'Structure syntaxique': 'NOM PRP NOM',
  'Fréquence (TF)': 12,


In [151]:
print('Après avoir calculé le log-likelihood ratio, on a retiré {} collocations qui n\'étaient pas statistiquement significatives.'.format(len(terms) - len(significant_coll)))
print('Ça représente environ {} % de nos n-grammes.'.format(round((len(terms) - len(significant_coll)) / len(terms) *100 )))

Après avoir calculé le log-likelihood ratio, on a retiré 0 collocations qui n'étaient pas statistiquement significatives.
Ça représente environ 0 % de nos n-grammes.


In [152]:
df = DataFrame(significant_coll).sort_values(by = "Fréquence (TF)", ascending=False).drop_duplicates()

# On veut faire join pour tous les termes[collocation]
def join_term(x):
    if type(x) == tuple:
        return " ".join(x).replace("' ", "'").replace("( ", "(").replace(" )", ")")
    else:
        return x

df['Terme'] = df['Terme'].apply(lambda x: join_term(x))

if lng == 'en':
    acteur = acteur + '_' + lng

output_path = path.join('../04-filtrage/output/', acteur + '_candidate-terms.csv') 

df.to_csv(output_path)

df = df.drop(columns=['p-value'])

### **Filtrage - Fréquence documentaire**

In [153]:
dfs = {term: len([doc for doc in text if term in doc]) for term in df['Terme'].tolist()}

In [154]:
max_df = round(0.95 * nb_docs)        # Pour rejeter les termes qui se retrouvent dans plus de 95% des documents 
min_df = round(0.01 * nb_docs)         # Pour rejeter les termes qui se retrouvent dans moins de 1% des documents
max_df

694

In [155]:
min_df

7

In [156]:
max_df_observed = dfs[max(dfs, key=dfs.get)] # Voir quelle est la fréquence documentaire maximale qu'on retrouve
max_df_observed

141

In [157]:
max_df_to_remove = {term:df for term,df in dfs.items() if df > max_df}
max_df_to_remove

{}

In [158]:
dfs = {term:df for term,df in dfs.items() if df < max_df and df > min_df} # Si df = 0, c'est un artefact créé par le prétraitement lui-même

Porter attention aux termes qui sortent ici - en comptant la fréquence documentaire, techniquement on ne devrait pas retrouver de 0

In [159]:
zeros =  {term:df for term,df in dfs.items() if df == 0}  
print(len(zeros))
zeros 

0


{}

In [160]:
dfs = DataFrame(list(dfs.items()),columns = ['Terme','Fréquence documentaire (DF)']) 

dfs

,Terme,Fréquence documentaire (DF)
0,santé mentale,141
1,maison de naissance,29
2,hôpital général,89
3,hôpital général juif,87
4,services sociaux,105
...,...,...
599,réseau d'éclaireur,24
600,services axés,8
601,camps de jour,9
602,réception des demandes de service,8


In [161]:
df = df.merge(dfs, on='Terme').drop_duplicates()

df.sort_values(['Fréquence (TF)'], 
            axis=0,
            ascending=[False], 
            inplace=True)

In [162]:
df

,Terme,Structure syntaxique,Fréquence (TF),LLR,Fréquence documentaire (DF)
0,santé mentale,NOM ADJ,437,-inf,141
1,maison de naissance,NOM PRP NOM,274,-inf,29
2,hôpital général,NOM ADJ,226,-inf,89
3,hôpital général juif,NOM ADJ ADJ,221,-inf,87
4,services sociaux,NOM ADJ,191,-inf,105
...,...,...,...,...,...
599,réseau d'éclaireur,NOM PRP NOM,6,84.233210,24
600,services axés,NOM VER:pper,6,46.652530,8
601,camps de jour,NOM PRP NOM,6,80.090126,9
602,réception des demandes de service,NOM PRP:det NOM PRP NOM,6,67.315136,8


In [163]:
list_terms = df['Terme'].tolist()

### **KWIC (Keyword in Context)**
Termes d'intérêt : 
- « Programme »
- « Plan »
- « Service(s) de » 
- « Intervenant(e) en »
- « Professionnel de »
- « Institut (du/de) »
- « Groupe de recherche en »
- « Personne »
- « Infirmière (en) »

In [164]:
# Dans notre cas on veut que ça débute par le mot-clé donc le contexte est un peu plus simple
# penser à généraliser avec des expressions régulières
# kw = ['programme', 'plan ', 'service', 'intervenant', 'infirmière en', 'institut', 'groupe de recherche', 'personne', 'maladie']

# ngrammes_kwic = [" ".join([t[0] for t in ng]).replace("' ", "'").replace("( ", "(").replace(" )", ")") for ng in ngrammes]

In [165]:
# ngrammes_kwic = [t for t in ngrammes_kwic if not 'STOP' in t]

In [166]:
# extrant = DataFrame(columns=['Mot-clé','Concordance', 'Fréquence (TF)'])
# kwic = {w : [] for w in kw} 

In [167]:
# for t in ngrammes_kwic: # on pourrait aussi chercher dans les terms, mais on perd certains termes d'intérêt avec le filtrage syntaxique
#     for w in kw:
#         if t.startswith(w):
#             kwic[w].append(t)

In [168]:
# kwic = {term: FreqDist(kwic[term]) for term in kwic}

In [169]:
# for term in kw:
#     df_kw = DataFrame(kwic[term].items(), columns=['Concordance', "Fréquence (TF)"])
#     df_kw.sort_values(["Fréquence (TF)"], 
#         axis=0,
#         ascending=[False], 
#         inplace=True)

#     df_kw.insert(0, 'Mot-clé', term)
#     extrant = concat([extrant, df])


# extrant = extrant[extrant['Fréquence (TF)'] > 30] 

# # file_path = '../04-filtrage/output/'
# # file_path = path.join(file_path, acteur, tag)


# # extrant.to_csv(file_path + '_KWIC' +'.csv')

# extrant

### **Extraction de termes MeSH**

In [170]:
df['isMeSHTerm']= False # On set à False puis on va changer pour True si on trouve le terme
df['MeSHID'] = None
df['MesH_prefLabel_fr'] = None
df['MesH_prefLabel_en'] = None

In [171]:
from nltk.tokenize import MWETokenizer
file_path = '../04-filtrage/MeSH/mesh-fr.txt'

with open (file_path, 'r', encoding='utf-8') as f:
    mesh = [tuple(tokenizer_re.tokenize(w)) for w in f.readlines()]
    tokenizer_mesh = MWETokenizer(mesh, separator= ' ')
    mesh = [tokenizer_mesh.tokenize(w)[0].lower() for w in mesh]
    mesh = [w for w in mesh if len(w.split()) > 1] # On ne retient que les termes complexes
    #mesh = [tuple(t.strip('.').lower().split()) for t in f.readlines()]

In [172]:
extr_mesh = tokenizer_mesh.tokenize(list_terms)

In [173]:
for t in extr_mesh:
    if t in mesh:
        df.loc[df['Terme'] == t, 'isMeSHTerm'] = True
df

,Terme,Structure syntaxique,Fréquence (TF),LLR,Fréquence documentaire (DF),isMeSHTerm,MeSHID,MesH_prefLabel_fr,MesH_prefLabel_en
0,santé mentale,NOM ADJ,437,-inf,141,True,None,None,None
1,maison de naissance,NOM PRP NOM,274,-inf,29,False,None,None,None
2,hôpital général,NOM ADJ,226,-inf,89,True,None,None,None
3,hôpital général juif,NOM ADJ ADJ,221,-inf,87,False,None,None,None
4,services sociaux,NOM ADJ,191,-inf,105,True,None,None,None
...,...,...,...,...,...,...,...,...,...
599,réseau d'éclaireur,NOM PRP NOM,6,84.233210,24,False,None,None,None
600,services axés,NOM VER:pper,6,46.652530,8,False,None,None,None
601,camps de jour,NOM PRP NOM,6,80.090126,9,False,None,None,None
602,réception des demandes de service,NOM PRP:det NOM PRP NOM,6,67.315136,8,False,None,None,None


In [174]:
# Termes MeSH présents dans notre corpus 
df[df['isMeSHTerm'] == True]

,Terme,Structure syntaxique,Fréquence (TF),LLR,Fréquence documentaire (DF),isMeSHTerm,MeSHID,MesH_prefLabel_fr,MesH_prefLabel_en
0,santé mentale,NOM ADJ,437,-inf,141,True,None,None,None
2,hôpital général,NOM ADJ,226,-inf,89,True,None,None,None
4,services sociaux,NOM ADJ,191,-inf,105,True,None,None,None
6,travailleur social,NOM ADJ,102,1455.830157,51,True,None,None,None
9,soins de santé,NOM PRP NOM,81,628.565775,55,True,None,None,None
12,professionnels de la santé,NOM PRP DET:ART NOM,73,750.812041,58,True,None,None,None
15,éthique clinique,NOM ADJ,70,618.344382,14,True,None,None,None
17,santé publique,NOM ADJ,67,627.692192,44,True,None,None,None
21,soins infirmiers,NOM NOM,60,652.139141,46,True,None,None,None
25,médecins de famille,NOM PRP NOM,57,745.799740,29,True,None,None,None


### **Extraction de termes existant dans la taxonomie**

In [175]:
df['isTaxoTerm']= 'False' # On set à False puis on va changer pour True si on trouve le terme

In [176]:
file_path = '../04-filtrage/default_taxo_labels.csv'

with open(file_path, 'r', encoding='utf-8') as f:
    default = read_csv(f, sep=';')
    taxo_terms = list(dict.fromkeys([str(t).strip().lower() for t in default['Label'].tolist()]))

taxo_terms

['1 on 1 support',
 '10 weeks and less ultrasound',
 '12-step facilitation therapy',
 '12-step programs',
 '18f pet bone scan',
 '18f bone pet',
 '18f bone scan',
 '1st generation antipsychotics',
 '2019 ncov testing',
 '2d doppler echocardiography',
 '2d echocardiography',
 '2d ultrasound',
 '2nd generation antipsychotics',
 '3d imaging',
 '3d sonography',
 '3d dental imaging',
 '3d echocardiography',
 '3d fetal ultrasound',
 '3d mammography',
 '3d prenatal ultrasound',
 '3d reconstruction',
 '46, xx disorder of sex development',
 '46,xy disorder of sex development',
 '4d echocardiography',
 '4d prenatal ultrasound',
 'a1c diabetes test',
 'a1c test',
 'abo + rh pnl bld',
 'abo and rh group in cord blood',
 'abpm device',
 'acl reconstruction surgery',
 'acth deficiency',
 'acth stimulation test',
 'adhd assessment',
 'adhd diagnostic',
 'adhd evaluation',
 'adhd testing',
 'adn foetal circulant',
 'aids',
 'aids nephropathy',
 'aids serodiagnosis',
 'aids-associated nephropathy',
 'a

In [177]:
for t in df['Terme'].tolist():
    if t in taxo_terms:
        df.loc[df['Terme'] == t, 'isTaxoTerm'] = True
df[df['isTaxoTerm'] == True]

,Terme,Structure syntaxique,Fréquence (TF),LLR,Fréquence documentaire (DF),isMeSHTerm,MeSHID,MesH_prefLabel_fr,MesH_prefLabel_en,isTaxoTerm
0,santé mentale,NOM ADJ,437,-inf,141,True,None,None,None,True
1,maison de naissance,NOM PRP NOM,274,-inf,29,False,None,None,None,True
2,hôpital général,NOM ADJ,226,-inf,89,True,None,None,None,True
4,services sociaux,NOM ADJ,191,-inf,105,True,None,None,None,True
5,médecin de famille,NOM PRP NOM,113,1498.614814,41,False,None,None,None,True
6,travailleur social,NOM ADJ,102,1455.830157,51,True,None,None,None,True
17,santé publique,NOM ADJ,67,627.692192,44,True,None,None,None,True
21,soins infirmiers,NOM NOM,60,652.139141,46,True,None,None,None,True
26,médecine familiale,NOM ADJ,56,695.151660,33,True,None,None,None,True
32,services de santé,NOM PRP NOM,52,287.255745,42,True,None,None,None,True


### **Mapping to MeSH IDs**

Lire le fichier XML contenant le MeSH Bilingue

Le MeSH 2019 comprend 29 351 descripteurs  
https://mesh.inserm.fr/FrenchMesh/presentation.htm

In [178]:
import xml.etree.ElementTree as ET
import re
import pandas as pd
base_path = '../04-filtrage/MeSH/'

tree = ET.parse(base_path + 'fredesc2019.xml')
root = tree.getroot()

def flatten(l):
    return [item for sublist in l for item in sublist]

data_mesh = [{'mesh_id' : x.find('DescriptorUI').text.strip('\n'), \
         'label_fr' : x.find('DescriptorName').find('String').text.split('[')[0], \
         'label_en' : x.find('DescriptorName').find('String').text.split('[')[1].strip(']'), \
         'synonymes (en/fr)' : flatten([[term.find('String').text for term in concept.find('TermList').findall('Term')] for concept in x.find('ConceptList').findall('Concept')]) \
         } for x in root.findall('DescriptorRecord')]

In [179]:
# for t in df[df['isMeSHTerm'] == True]['Terme'].tolist():
#     for d in data_mesh:
#         if t == str(d['label_fr']).lower():
#             df.loc[df['Terme'] == t, 'MeSHID'] = d['mesh_id']
#             df.loc[df['Terme'] == t, 'MesH_prefLabel_fr'] = d['label_fr']
#             df.loc[df['Terme'] == t, 'MesH_prefLabel_en'] = d['label_en']

In [180]:
for t in df[df['isMeSHTerm'] == True]['Terme'].tolist():
    for d in data_mesh:
        if t in [str(x).lower() for x in d['synonymes (en/fr)']]:
            df.loc[df['Terme'] == t, 'MeSHID'] = d['mesh_id']
            df.loc[df['Terme'] == t, 'MesH_prefLabel_fr'] = d['label_fr']
            df.loc[df['Terme'] == t, 'MesH_prefLabel_en'] = d['label_en']

df = df[['Terme', 'Structure syntaxique',	'Fréquence (TF)', 'Fréquence documentaire (DF)', 'LLR', 'isMeSHTerm', 'isTaxoTerm', 'MeSHID', 'MesH_prefLabel_fr', 'MesH_prefLabel_en']]

In [181]:
df[df['isMeSHTerm'] == True]

,Terme,Structure syntaxique,Fréquence (TF),Fréquence documentaire (DF),LLR,isMeSHTerm,isTaxoTerm,MeSHID,MesH_prefLabel_fr,MesH_prefLabel_en
0,santé mentale,NOM ADJ,437,141,-inf,True,True,D008603,Santé mentale,Mental Health
2,hôpital général,NOM ADJ,226,89,-inf,True,True,D006769,Hôpitaux généraux,"Hospitals, General"
4,services sociaux,NOM ADJ,191,105,-inf,True,True,D012947,Services sociaux et travail social (activité),Social Work
6,travailleur social,NOM ADJ,102,51,1455.830157,True,True,D000068217,Travailleurs sociaux,Social Workers
9,soins de santé,NOM PRP NOM,81,55,628.565775,True,False,D003695,Prestations des soins de santé,Delivery of Health Care
12,professionnels de la santé,NOM PRP DET:ART NOM,73,58,750.812041,True,False,D006282,Personnel de santé,Health Personnel
15,éthique clinique,NOM ADJ,70,14,618.344382,True,False,D026690,Éthique clinique,"Ethics, Clinical"
17,santé publique,NOM ADJ,67,44,627.692192,True,True,D011634,Santé publique,Public Health
21,soins infirmiers,NOM NOM,60,46,652.139141,True,True,D009732,Soins infirmiers,Nursing Care
25,médecins de famille,NOM PRP NOM,57,29,745.799740,True,False,D010821,Médecins de famille,"Physicians, Family"


In [182]:
df.insert(0, 'Corpus', acteur)
df

,Corpus,Terme,Structure syntaxique,Fréquence (TF),Fréquence documentaire (DF),LLR,isMeSHTerm,isTaxoTerm,MeSHID,MesH_prefLabel_fr,MesH_prefLabel_en
0,ciusss_centreouest,santé mentale,NOM ADJ,437,141,-inf,True,True,D008603,Santé mentale,Mental Health
1,ciusss_centreouest,maison de naissance,NOM PRP NOM,274,29,-inf,False,True,None,None,None
2,ciusss_centreouest,hôpital général,NOM ADJ,226,89,-inf,True,True,D006769,Hôpitaux généraux,"Hospitals, General"
3,ciusss_centreouest,hôpital général juif,NOM ADJ ADJ,221,87,-inf,False,False,None,None,None
4,ciusss_centreouest,services sociaux,NOM ADJ,191,105,-inf,True,True,D012947,Services sociaux et travail social (activité),Social Work
...,...,...,...,...,...,...,...,...,...,...,...
599,ciusss_centreouest,réseau d'éclaireur,NOM PRP NOM,6,24,84.233210,False,False,None,None,None
600,ciusss_centreouest,services axés,NOM VER:pper,6,8,46.652530,False,False,None,None,None
601,ciusss_centreouest,camps de jour,NOM PRP NOM,6,9,80.090126,False,False,None,None,None
602,ciusss_centreouest,réception des demandes de service,NOM PRP:det NOM PRP NOM,6,8,67.315136,False,False,None,None,None


In [183]:
if lng == 'en':
    acteur = acteur + '_' + lng

output_path = path.join('../04-filtrage/output/', acteur + '_candidate-terms.csv') 

df.to_csv(output_path)

In [184]:
df[df['isMeSHTerm'] == True]

,Corpus,Terme,Structure syntaxique,Fréquence (TF),Fréquence documentaire (DF),LLR,isMeSHTerm,isTaxoTerm,MeSHID,MesH_prefLabel_fr,MesH_prefLabel_en
0,ciusss_centreouest,santé mentale,NOM ADJ,437,141,-inf,True,True,D008603,Santé mentale,Mental Health
2,ciusss_centreouest,hôpital général,NOM ADJ,226,89,-inf,True,True,D006769,Hôpitaux généraux,"Hospitals, General"
4,ciusss_centreouest,services sociaux,NOM ADJ,191,105,-inf,True,True,D012947,Services sociaux et travail social (activité),Social Work
6,ciusss_centreouest,travailleur social,NOM ADJ,102,51,1455.830157,True,True,D000068217,Travailleurs sociaux,Social Workers
9,ciusss_centreouest,soins de santé,NOM PRP NOM,81,55,628.565775,True,False,D003695,Prestations des soins de santé,Delivery of Health Care
12,ciusss_centreouest,professionnels de la santé,NOM PRP DET:ART NOM,73,58,750.812041,True,False,D006282,Personnel de santé,Health Personnel
15,ciusss_centreouest,éthique clinique,NOM ADJ,70,14,618.344382,True,False,D026690,Éthique clinique,"Ethics, Clinical"
17,ciusss_centreouest,santé publique,NOM ADJ,67,44,627.692192,True,True,D011634,Santé publique,Public Health
21,ciusss_centreouest,soins infirmiers,NOM NOM,60,46,652.139141,True,True,D009732,Soins infirmiers,Nursing Care
25,ciusss_centreouest,médecins de famille,NOM PRP NOM,57,29,745.799740,True,False,D010821,Médecins de famille,"Physicians, Family"
